In [2]:
import petpy
import pandas as pd
import numpy as np
import random

In [3]:
petfinder=petpy.Petfinder(
    key='uTe0odJUWZhi3Lv0pyLteRLLJVv9Qv65GTH6zJDEBoEAlmd7kt', 
    secret='HGN1qte2FpD8TgaL9WtuZF9WDuPUq129RVxUYDR7')

In [5]:
# Debbie
dogs_test_wa=petfinder.animals(animal_type='dog',
                  status='adopted',
                  before_date='2023-01-01', 
                  after_date='2022-12-01',
                  location = "WA",
                  pages=None,
                  return_df=True)

In [7]:
dogs_test_wa.to_csv("dog_test_wa.csv")

In [ ]:
dogs_18_months=petfinder.animals(animal_type='dog',
                  status='adopted',
                  before_date='2023-01-01', 
                  after_date='2022-07-01', 
                  pages=None,
                  return_df=True)

In [ ]:
dogs_18_months.to_csv('dogs_18_months.csv', index=False)

In [ ]:
cats_18_months=petfinder.animals(animal_type='cat',
                  status='adopted',
                  before_date='2023-07-01', 
                  after_date='2022-01-01', 
                  pages=None,
                  return_df=True)

In [ ]:
cats_18_months.to_csv('cats_18_months.csv', index=False)

In [27]:
dogs = pd.read_csv('dogs_18_months.csv')
cats = pd.read_csv('cats_18_months.csv')

In [13]:
dogs_US = dogs[dogs['contact.address.country'] == 'US'].copy()
cats_US = cats[cats['contact.address.country'] == 'US'].copy()

In [14]:
dogs_US.drop_duplicates(subset=['id'], inplace=True)
cats_US.drop_duplicates(subset=['id'], inplace=True)

In [15]:
US = pd.concat([dogs_US, cats_US])

In [16]:
US['published_at']=pd.to_datetime(US['published_at'])
US['status_changed_at']=pd.to_datetime(US['status_changed_at'])

In [17]:
US['days_in_shelter']=US['status_changed_at']-US['published_at']
US['days_in_shelter']=US['days_in_shelter']/np.timedelta64(1,'D')
US['days_in_shelter']=US['days_in_shelter'].round()

In [18]:
US['breed'] = ''
# Extract primary and secondary breed from 'breeds' column
for i, row in US.iterrows():
    primary_breed = row['breeds.primary']
    secondary_breed = row['breeds.secondary']
    mixed = row['breeds.mixed']
    unknown_breed = row['breeds.unknown']
    
    if mixed:
        US.at[i, 'breed'] = 'Mixed'
    elif pd.notna(primary_breed):
        US.at[i, 'breed'] = primary_breed
    elif pd.notna(unknown_breed):
        US.at[i, 'breed'] = unknown_breed
    elif pd.notna(secondary_breed):
        US.at[i, 'breed'] = secondary_breed
    else:
        US.at[i, 'breed'] = 'Unknown'

In [19]:
# Select relevant columns
US = US[['id', 'type', 'species', 'age', 'gender', 'size', 'coat', 'tags', 'name', 'description', 'status', 
         'published_at', 'status_changed_at', 'days_in_shelter', 'breed',
         'colors.primary', 'attributes.spayed_neutered', 'attributes.house_trained', 'attributes.special_needs', 
         'attributes.shots_current', 'environment.dogs']]

In [20]:
US.to_csv('US.csv', index=False)

In [23]:
US.shape
#this data has been saved as '18_adopted_months' on the google drive

(156361, 21)